**1. Смотрим на Hadoop Distributed File System**

В рамках этой части вам нужно будет обращаться к HDFS с помощью CLI, разместить файлы для следующих заданий в распределеннй файловой системе и выполнить несколько преобразований над ними.

Для работы файлы можно скачать по следующим ссылкам:
- Логи посещения сайтов юзерами за некоторый промежуток времени [ссылка](https://drive.google.com/file/d/1WXyq5WVSWwJYXPuH4kyAJ5mrR3XgfO_H/view?usp=sharing)

Разместите их в нашем внутреннем файловом хранилище с помощью HDFS CLI, для дальнейшего удобства под каждый файл стоит создать каталог с простым и понятным именем, разместить сами файлы в разных каталогах.

Набор комманд, которые вам могут в этом помочь, доступны [здесь](https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/FileSystemShell.html)

В ячейках ниже должен быть полный набор комманд ваших обращей к консоли

In [ ]:
## вы можете обращаться к консоли из ноутбука таким способом
!hdfs dfs -ls /

Found 3 items
drwxr-xr-x   - root supergroup          0 2024-06-01 07:40 /input
drwxr-xr-x   - root supergroup          0 2024-06-01 07:42 /output
drwxr-xr-x   - root supergroup          0 2024-06-01 07:34 /tmp


In [ ]:
%%bash
## или же использовать для этого меджик строчку в ячейке %%bash, как вам будет удобнее

hdfs dfs -ls /

Found 3 items
drwxr-xr-x   - root supergroup          0 2024-06-01 07:40 /input
drwxr-xr-x   - root supergroup          0 2024-06-01 07:42 /output
drwxr-xr-x   - root supergroup          0 2024-06-01 07:34 /tmp


In [ ]:
!hdfs dfs -ls -R -h /input

-rw-r--r--   1 root supergroup         24 2024-06-01 07:40 /input/input.txt


In [ ]:
## ваше решение здесь

**2. Решаем задачи MapReduce**

**2.1 Подсчет слов в тексте**

В рамках данного задания вам нужно подсчитать кол-во слов в тексте Библии (файл приложен к ДЗ в чате тг), то есть необходимо реализовать базовый функционал утилиты word count.

**Важно** - подсчитывайте число только тех слов, длина которых больше 4 символов. Проводить процесс удаления знаков препинания и прочих символов **не нужно**

Ниже вам представлены ячейки, в которых вы должны описать структуру маппера/редьсюера и ниже вызвать их в bash-скрипте запуска MR-таски

In [11]:
%%writefile mapper.py

import sys

for line in sys.stdin:
    line = line.strip()
    if line:
        words = line.split()
        for word in words:
            if len(word) > 4:
                print(f"{word.lower()}\t1")

Overwriting mapper.py


In [12]:
%%writefile reducer.py

import sys
from collections import defaultdict

word_counts = defaultdict(int)

for line in sys.stdin:
    line = line.strip()
    if line:
        word, count = line.split('\t')
        word_counts[word] += int(count)

for word, count in word_counts.items():
    print(f"{word}\t{count}")

Overwriting reducer.py


В качестве проверки ваших python-скриптов до запуска MR таски можно произвести их запуск через консольные команды

Тогда наша задача не будет выполняться через датаноды, а только на локальной машине, но в случае ошибок в скриптах вы увидите их и сможете исправить

In [13]:
%%bash
## пример запуска скриптов на неймноде для проверки их работы

cat bible.txt | python3 mapper.py | sort -k1,1 | python3 reducer.py | sort -k2 -nr | head -n 20

shall	9733
which	4244
their	3859
there	2008
before	1722
lord,	1613
against	1596
shalt	1589
children	1560
said,	1556
them,	1549
saying,	1272
house	1222
every	1208
people	1194
because	1178
thee,	1170
these	1147
saith	1135
after	1128


In [8]:
! hdfs dfs -ls

Found 4 items
-rw-r--r--   1 ubuntu hadoop    4047392 2025-04-16 18:01 bible.txt
drwxr-xr-x   - ubuntu hadoop          0 2025-04-16 17:09 input
drwxr-xr-x   - ubuntu hadoop          0 2025-04-16 17:34 word_count_output
drwxr-xr-x   - ubuntu hadoop          0 2025-04-16 18:04 word_count_task


Как только в данной проверке вы получите успешный и корректный результат, можете запустить Map Reduce в ячейке ниже

In [14]:
%%bash
## шаблон для запуска MR таски

# обязательная чистка директории, куда будем складывать результат отрабоки mr
hdfs dfs -rm -r /user/ubuntu/word_count_task || true

# запус mr таски с указанием пути до нужного jar
hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="word-count" \
    -files ./mapper.py,./reducer.py \
    -mapper "python3 mapper.py" \
    -reducer "python3 reducer.py" \
    -input /user/ubuntu/bible.txt \
    -output /user/ubuntu/word_count_task

Deleted /user/ubuntu/word_count_task
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob6250693543778962723.jar tmpDir=null


2025-04-16 19:08:45,112 INFO client.RMProxy: Connecting to ResourceManager at rc1b-dataproc-m-np3jage98jg69p5f.mdb.yandexcloud.net/10.129.0.25:8032
2025-04-16 19:08:45,340 INFO client.AHSProxy: Connecting to Application History server at rc1b-dataproc-m-np3jage98jg69p5f.mdb.yandexcloud.net/10.129.0.25:10200
2025-04-16 19:08:45,376 INFO client.RMProxy: Connecting to ResourceManager at rc1b-dataproc-m-np3jage98jg69p5f.mdb.yandexcloud.net/10.129.0.25:8032
2025-04-16 19:08:45,376 INFO client.AHSProxy: Connecting to Application History server at rc1b-dataproc-m-np3jage98jg69p5f.mdb.yandexcloud.net/10.129.0.25:10200
2025-04-16 19:08:45,572 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1744802518607_0021
2025-04-16 19:08:45,898 INFO mapred.FileInputFormat: Total input files to process : 1
2025-04-16 19:08:45,984 INFO mapreduce.JobSubmitter: number of splits:30
2025-04-16 19:08:46,145 INFO mapreduce.JobSubmitter: Submitting 

In [8]:
%%bash
hdfs dfs -ls /user/ubuntu/

Found 2 items
drwxr-xr-x   - ubuntu hadoop          0 2025-04-16 17:09 /user/ubuntu/input
drwxr-xr-x   - ubuntu hadoop          0 2025-04-16 17:34 /user/ubuntu/word_count_output


In [15]:
%%bash

hdfs dfs -cat /user/ubuntu/word_count_task/*

(being	1
(touch	1
aaron's	31
abia:	1
abiah.	1
abiah:	1
abida,	1
abigail	12
abigail,	3
abihu	3
abihu,	9
abihud,	1
abijah:	1
abode.	2
abominable?	1
abomination	55
abominations;	3
abounding	3
about	434
about,	84
abraham;	4
abram	35
abram,	14
absalom.	15
absalom:	4
accepted	22
accepted,	3
accompanying	1
accomplish	12
account	12
accusers,	2
achzib:	1
acquaintance,	1
acts.	1
acts:	2
adina	1
adithaim,	1
adonijah?	1
adoniram	2
adulterers	2
adultress	1
advantage.	1
adversary	18
adversities	1
affrighted;	1
afoot.	1
afraid,	38
after;	3
again;	17
against	1596
aground;	1
aharah	1
ahasbai,	1
ahasuerus.	4
ahaziah,)	1
ahijah	13
ahikam.	1
ahikam:	1
ahiramites:	1
ahitub	3
aholah,	1
aija,	1
alamoth;	1
alienated,	1
alike;	1
alleging,	1
allied	1
already;	1
altereth	2
always,	8
amalek.	2
amalek:	4
amber.	1
amethyst.	3
ammon;	4
ammonites	6
amongst	2
anak,	5
ananias	6
anathoth,	11
anger.	24
anger:	7
angry	30
angry,	8
anoint:	1
anointest	1
answer?	1
answerest?	1
antichrist,	2
antichrists;	1
apart;	5
appeareth?

Мониторить процесс работы таски можно на nodemanager по порту 8088 (уже прокинут в конфиге), там будет UI, в котором будет видно вашу запущенную задачу и её статус.

Результат работы скрипта должен выглядеть следующим образом (вывод тестовый):

```bash
word count
abtr 6852
btoad 4237
stress 1932
zen 1885
```

In [16]:
%%bash
## запустите эту команду, чтобы вывести счетчик определенных слов, которые мы указали на grep
## Это нам будет необходимо для визуального анализа результата работы вашего скрипта
## в sort можете указать тот разделитель колонок, с которым у вас результат выплевывает редьюсер

hdfs dfs -cat /user/ubuntu/word_count_task/* | grep  -E 'lord\.|god\.|pray\.' | sort -t$'\t' -k2.2nr | head -n 3

lord.	674
pray.	5


**2.2 Решаем задачу поиска самых посещаемых сайтов**

В данном задании нужно поработать с логом данных о посещении юзерами различных сайтов.
Формат данных: `url;временная метка`. Вам нужно вывести топ 5 сайтов по посещаемости в каждую из дат, которая представлена в наших данных.

Результат работы скрипта должен выглядеть следующим образом:

```bash
date        site                            count
2024-05-25  https://gonzales-bautista.com/  987
2024-05-25  https://smith.com/              654
2024-05-25  https://www.smith.com/          321
```

**Рекомендации**

1. Вам могу пригодиться дополнительные параметры mr таски, отвечающие за настройку шаффла, и правил сортировки ключей внутри него. Почитать о примерах их использования можно [здесь](https://hadoop.apache.org/docs/current/hadoop-streaming/HadoopStreaming.html#More_Usage_Examples).

2. Не рекомендуем использовать `\t` в качестве символа разделителя для сложного ключа (потому что по дефолту таб используется для разделения колонок данных, и ключом в таком случае будет только первая колонка до таба). Если вы будете собирать сложный ключ для нужной вам сортировки данных, лучше всего будет использовать другие симловы, к примеру `+, =`.

3. Возможно, у вас не получится решить данную задачу за одну mr таску, тогда вы просто описываете в решении скрипты ваших мапперов, редьюсеров под каждую из mr тасок, которые вам нужно запустить для получения нужного результата.

**Важно** помнить, что любой маппер и редьюсер должен работать за O(1) памяти, и если вы будете создавать какой-то список, куда будете складывать какие-то данные, то он не должен быть размера O(n). Если такой момент в вашем решении будет, пожалуйста, поясните его текстово, что с вашими переменными все ок, и у них нет размера O(n).

In [16]:
%%writefile mapper1.py
import sys
from datetime import datetime

for line in sys.stdin:
    try:
        url, timestamp = line.strip().split(';')
        # Преобразуем timestamp в дату (формат YYYY-MM-DD)
        date = datetime.fromtimestamp(float(timestamp)).strftime('%Y-%m-%d')
        # Ключ: дата + url, значение: 1
        print(f"{date}\t{url}\t1")
    except:
        # Пропускаем некорректные строки
        continue

Overwriting mapper1.py


In [17]:
%%writefile reducer1.py
import sys
from collections import defaultdict

def main():
    # Словарь для хранения: {дата: {url: count}}
    date_stats = defaultdict(lambda: defaultdict(int))
    
    for line in sys.stdin:
        try:
            date, url, count = line.strip().split('\t')
            date_stats[date][url] += int(count)
        except:
            continue
    
    # Выводим топ-5 для каждой даты
    for date in sorted(date_stats.keys()):
        # Сортируем URL по посещаемости (по убыванию)
        top_sites = sorted(date_stats[date].items(),
                          key=lambda x: x[1],
                          reverse=True)[:5]
        
        for url, count in top_sites:
            print(f"{date}\t{url}\t{count}")

if __name__ == "__main__":
    main()

Overwriting reducer1.py


In [24]:
%%bash
# Выводим заголовок
echo -e "date\tsite\tcount"
# Обрабатываем данные
cat sites.csv | python3 mapper1.py | sort -k1,1 | python3 reducer1.py | sort -k1,1 -k3,3nr | awk '$1 != prev {prev=$1; cnt=1} cnt <=5 {print; cnt++}'

date	site	count


In [51]:
! head -n 5 sites.csv | python3 mapper1.py | sort -t$'\t' -k1,1

In [10]:
! hdfs dfs -ls /user/ubuntu

Found 3 items
-rw-r--r--   1 ubuntu hadoop    4047392 2025-04-16 18:01 /user/ubuntu/bible.txt
drwxr-xr-x   - ubuntu hadoop          0 2025-04-16 17:09 /user/ubuntu/input
drwxr-xr-x   - ubuntu hadoop          0 2025-04-16 17:34 /user/ubuntu/word_count_output


In [12]:
! hdfs dfs -put sites.csv /user/ubuntu/sites.csv

put: `/user/ubuntu/sites.csv': File exists


In [18]:
! hdfs dfs -head /user/ubuntu/sites.csv


















https://gonzales-bautista.com/;2024-05-28 17:41:40.8

In [11]:
! hdfs dfs -ls /user/ubuntu/

Found 5 items
-rw-r--r--   1 ubuntu hadoop    4047392 2025-04-16 18:01 /user/ubuntu/bible.txt
drwxr-xr-x   - ubuntu hadoop          0 2025-04-16 17:09 /user/ubuntu/input
-rw-r--r--   1 ubuntu hadoop   36443383 2025-04-16 21:25 /user/ubuntu/sites.csv
drwxr-xr-x   - ubuntu hadoop          0 2025-04-16 21:27 /user/ubuntu/url_task_2
drwxr-xr-x   - ubuntu hadoop          0 2025-04-16 17:34 /user/ubuntu/word_count_output


In [19]:
! hdfs dfs -cat /user/ubuntu/url_task_2/* | grep -E '2024-05-28|2024-06-02|2024-05-30' | column -t -s$'\t'

http://abbott-brown.com/;2024-05-28                1
http://acevedo-keith.biz/;2024-05-28               1
http://acevedo-singh.com/;2024-05-28               1
http://acevedo.info/;2024-06-02                    1
http://acosta-harper.com/;2024-05-30               1
http://acosta-jenkins.com/;2024-05-30              2
http://acosta-spencer.com/;2024-05-30              17
http://acosta.info/;2024-05-30                     2
http://adams-ball.com/;2024-05-30                  1
http://adams-brown.com/;2024-05-28                 1
http://adams-brown.info/;2024-05-30                1
http://adams-jackson.info/;2024-05-28              1
http://adams-perez.com/;2024-05-28                 1
http://adams.biz/;2024-05-30                       5
http://aguilar-reed.com/;2024-05-30                1
http://aguirre-herrera.com/;2024-05-28             1
http://alexander-bailey.org/;2024-05-30            1
http://alexander-tucker.org/;2024-05-30            1
http://alexander.com/;2024-05-30             

http://weber-mcbride.biz/;2024-05-30               1
http://weber.com/;2024-05-28                       11
http://weber.info/;2024-05-30                      4
http://webster-dominguez.biz/;2024-05-28           2
http://webster.com/;2024-05-30                     4
http://weiss-turner.com/;2024-05-30                1
http://wells.com/;2024-05-28                       13
http://west-cook.info/;2024-05-28                  1
http://west-lopez.com/;2024-05-30                  1
http://west-roach.com/;2024-05-28                  1
http://west-sanders.net/;2024-05-30                1
http://west.com/;2024-05-28                        11
http://west.net/;2024-05-28                        7
http://whitaker-brewer.net/;2024-05-28             1
http://white-freeman.org/;2024-05-28               2
http://white-garcia.com/;2024-05-28                2
http://white-kemp.biz/;2024-05-28                  1
http://white-lane.com/;2024-05-30                  1
http://white-mays.biz/;2024-05-30          

http://melendez-gray.org/;2024-05-30               1
http://melendez.com/;2024-05-28                    5
http://melton-campbell.com/;2024-05-28             1
http://mendez-mitchell.com/;2024-05-28             1
http://mendez-paul.com/;2024-05-28                 1
http://mendez.biz/;2024-05-30                      2
http://mendoza-dawson.com/;2024-05-30              2
http://mendoza-miller.info/;2024-05-30             1
http://meyer-sanders.com/;2024-05-30               1
http://meyers-holmes.net/;2024-05-28               1
http://meyers.org/;2024-05-28                      1
http://miles-cain.biz/;2024-05-28                  1
http://miller-campos.info/;2024-05-28              4
http://miller-day.com/;2024-05-28                  1
http://miller-gill.com/;2024-05-28                 1
http://miller-hansen.net/;2024-05-30               2
http://miller-olson.com/;2024-05-30                1
http://miller-pittman.com/;2024-05-30              1
http://miller-sanders.info/;2024-05-30        

https://www.moore-henderson.com/;2024-05-30        1
https://www.moore-mason.com/;2024-05-30            1
https://www.moore-olson.com/;2024-05-28            1
https://www.moore-park.com/;2024-05-30             1
https://www.morales.com/;2024-05-30                10
https://www.morales.net/;2024-05-30                2
https://www.moran-morales.com/;2024-05-30          4
https://www.moran-stewart.net/;2024-05-28          2
https://www.moreno-hernandez.biz/;2024-05-28       1
https://www.moreno-hill.com/;2024-05-28            1
https://www.morgan-martinez.org/;2024-05-28        1
https://www.morgan-ramirez.com/;2024-05-30         1
https://www.morgan.biz/;2024-05-28                 3
https://www.morgan.info/;2024-05-30                7
https://www.morgan.org/;2024-05-30                 4
https://www.morris-lee.org/;2024-05-28             1
https://www.morris-mendoza.com/;2024-05-30         1
https://www.morris-sellers.net/;2024-05-30         2
https://www.morris.info/;2024-05-30          

https://sanchez-roberts.com/;2024-05-28            1
https://sanchez-thornton.com/;2024-05-30           1
https://sanchez-vaughn.com/;2024-05-28             1
https://sanchez.info/;2024-05-30                   6
https://sanders-howard.com/;2024-05-28             1
https://sanders.net/;2024-05-28                    1
https://sandoval.info/;2024-05-30                  1
https://sanford.net/;2024-05-30                    3
https://sanford.org/;2024-05-28                    1
https://santana-cordova.com/;2024-05-30            1
https://santiago-costa.com/;2024-05-30             1
https://saunders-gill.com/;2024-05-28              1
https://savage-walton.com/;2024-05-30              1
https://savage.org/;2024-05-28                     1
https://sawyer-hayes.com/;2024-05-28               2
https://schaefer-thompson.com/;2024-05-28          1
https://schaefer.com/;2024-05-30                   9
https://schaefer.net/;2024-05-30                   1
https://schmidt-flynn.net/;2024-05-28         

...

https://www.austin-carter.net/;2024-05-30          3
https://www.austin.org/;2024-05-28                 4
https://www.avery.com/;2024-05-28                  3
https://www.avila-johnson.com/;2024-05-28          1
https://www.avila-myers.biz/;2024-05-30            1
https://www.bailey-carrillo.com/;2024-05-28        1
https://www.bailey-kennedy.com/;2024-05-28         1
https://www.bailey-torres.com/;2024-05-30          4
https://www.bailey.net/;2024-05-30                 3
https://www.bailey.org/;2024-05-28                 3
https://www.baker-ramsey.net/;2024-05-28           1
https://www.baker-velasquez.com/;2024-05-30        1
https://www.baldwin.org/;2024-05-30                2
https://www.ball-adkins.com/;2024-05-30            1
https://www.ballard-miller.com/;2024-05-30         1
https://www.banks-west.com/;2024-05-28             1
https://www.banks.info/;2024-05-30                 3
https://www.banks.net/;2024-05-28                  1
https://www.barber-lee.org/;2024-05-30        

https://brewer.org/;2024-05-30                     5
https://bridges-mejia.biz/;2024-05-28              1
https://bright.com/;2024-05-30                     2
https://bright.net/;2024-05-30                     1
https://brock-castillo.biz/;2024-05-28             1
https://brock-thompson.net/;2024-05-30             1
https://brown-campbell.biz/;2024-05-28             1
https://brown-gould.com/;2024-05-30                1
https://brown-hays.com/;2024-05-30                 2
https://brown-johnson.info/;2024-05-28             1
https://brown-jones.info/;2024-05-28               1
https://brown-lawrence.net/;2024-05-30             1
https://brown-luna.biz/;2024-05-28                 1
https://brown-meyer.biz/;2024-05-28                1
https://brown-morris.info/;2024-05-30              2
https://brown-rogers.info/;2024-05-30              1
https://brown-santiago.org/;2024-05-30             1
https://brown-smith.biz/;2024-05-30                1
https://brown-stewart.net/;2024-05-28         

http://www.carter-kirby.net/;2024-05-30            1
http://www.carter-oliver.info/;2024-05-28          1
http://www.carter-turner.com/;2024-05-28           1
http://www.carter.biz/;2024-05-30                  6
http://www.case-flowers.com/;2024-05-30            1
http://www.casey-mejia.com/;2024-05-28             1
http://www.castaneda-jenkins.com/;2024-05-30       1
http://www.castaneda.com/;2024-05-28               43
http://www.castillo-garcia.com/;2024-05-30         1
http://www.castro-bruce.com/;2024-05-28            1
http://www.castro-kelly.info/;2024-05-30           1
http://www.castro-miller.com/;2024-05-28           1
http://www.castro-walker.com/;2024-05-30           6
http://www.cervantes.net/;2024-05-30               1
http://www.cervantes.org/;2024-05-28               2
http://www.chambers-berry.com/;2024-05-28          1
http://www.chambers-hernandez.info/;2024-05-28     1
http://www.chambers-johnson.net/;2024-05-30        2
http://www.chandler.com/;2024-05-30          

http://flynn.biz/;2024-05-30                       2
http://foley-malone.info/;2024-05-30               1
http://foley-wiley.com/;2024-05-28                 1
http://foley.biz/;2024-05-30                       1
http://ford-hines.info/;2024-05-28                 1
http://ford-lee.net/;2024-05-28                    1
http://ford-martin.com/;2024-05-30                 1
http://ford-tanner.net/;2024-05-28                 1
http://foster-arnold.com/;2024-05-28               1
http://foster-frederick.com/;2024-05-30            1
http://foster-hogan.org/;2024-05-28                1
http://foster-walsh.info/;2024-05-28               1
http://fowler.com/;2024-05-30                      4
http://fowler.net/;2024-05-30                      3
http://fox-brooks.com/;2024-05-28                  6
http://fox-kelly.com/;2024-05-30                   1
http://francis-crosby.com/;2024-05-30              1
http://francis.com/;2024-05-28                     3
http://frank.org/;2024-05-30                  

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
%%bash
## шаблон для запуска MR таски

# обязательная чистка директории, куда будем складывать результат отрабоки mr
hdfs dfs -rm -r /user/ubuntu/url_task_2 || true

# Запуск MapReduce задачи
hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="top-sites-by-date" \
    -D stream.num.map.output.key.fields=2 \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-k1,1 -k2,2" \
    -files ./mapper.py,./reducer.py \
    -mapper "python3 mapper.py" \
    -reducer "python3 reducer.py" \
    -input /user/ubuntu/sites.csv \
    -output /user/ubuntu/url_task_2

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob4333718582423655116.jar tmpDir=null


rm: `/user/ubuntu/url_task_2': No such file or directory
2025-04-16 21:26:58,259 INFO client.RMProxy: Connecting to ResourceManager at rc1b-dataproc-m-np3jage98jg69p5f.mdb.yandexcloud.net/10.129.0.25:8032
2025-04-16 21:26:58,491 INFO client.AHSProxy: Connecting to Application History server at rc1b-dataproc-m-np3jage98jg69p5f.mdb.yandexcloud.net/10.129.0.25:10200
2025-04-16 21:26:58,533 INFO client.RMProxy: Connecting to ResourceManager at rc1b-dataproc-m-np3jage98jg69p5f.mdb.yandexcloud.net/10.129.0.25:8032
2025-04-16 21:26:58,534 INFO client.AHSProxy: Connecting to Application History server at rc1b-dataproc-m-np3jage98jg69p5f.mdb.yandexcloud.net/10.129.0.25:10200
2025-04-16 21:26:58,761 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1744802518607_0025
2025-04-16 21:26:59,176 INFO mapred.FileInputFormat: Total input files to process : 1
2025-04-16 21:26:59,328 INFO mapreduce.JobSubmitter: number of splits:30
2025-04

In [20]:
%%bash
## запустите эту команду, чтобы вывести результат работы по определенным компаниям, которые мы указали на grep
## Это нам будет необходимо для визуального анализа результата работы вашего скрипта
## в sort можете указать тот разделитель колонок, с которым у вас результат выплевывает редьюсер
## укажите путь до той директории на hdfs, куда вы складывали результат

hdfs dfs -cat /user/ubuntu/url_task_2/* | grep -E '2024-05-28|2024-06-02|2024-05-30' | column -t -s$'\t' -n 20


column: 20: No such file or directory
cat: Unable to write to output stream.
cat: Unable to write to output stream.
cat: Unable to write to output stream.
cat: Unable to write to output stream.
cat: Unable to write to output stream.
cat: Unable to write to output stream.
cat: Unable to write to output stream.
cat: Unable to write to output stream.
cat: Unable to write to output stream.
cat: Unable to write to output stream.
cat: Unable to write to output stream.
cat: Unable to write to output stream.


CalledProcessError: Command 'b"## \xd0\xb7\xd0\xb0\xd0\xbf\xd1\x83\xd1\x81\xd1\x82\xd0\xb8\xd1\x82\xd0\xb5 \xd1\x8d\xd1\x82\xd1\x83 \xd0\xba\xd0\xbe\xd0\xbc\xd0\xb0\xd0\xbd\xd0\xb4\xd1\x83, \xd1\x87\xd1\x82\xd0\xbe\xd0\xb1\xd1\x8b \xd0\xb2\xd1\x8b\xd0\xb2\xd0\xb5\xd1\x81\xd1\x82\xd0\xb8 \xd1\x80\xd0\xb5\xd0\xb7\xd1\x83\xd0\xbb\xd1\x8c\xd1\x82\xd0\xb0\xd1\x82 \xd1\x80\xd0\xb0\xd0\xb1\xd0\xbe\xd1\x82\xd1\x8b \xd0\xbf\xd0\xbe \xd0\xbe\xd0\xbf\xd1\x80\xd0\xb5\xd0\xb4\xd0\xb5\xd0\xbb\xd0\xb5\xd0\xbd\xd0\xbd\xd1\x8b\xd0\xbc \xd0\xba\xd0\xbe\xd0\xbc\xd0\xbf\xd0\xb0\xd0\xbd\xd0\xb8\xd1\x8f\xd0\xbc, \xd0\xba\xd0\xbe\xd1\x82\xd0\xbe\xd1\x80\xd1\x8b\xd0\xb5 \xd0\xbc\xd1\x8b \xd1\x83\xd0\xba\xd0\xb0\xd0\xb7\xd0\xb0\xd0\xbb\xd0\xb8 \xd0\xbd\xd0\xb0 grep\n## \xd0\xad\xd1\x82\xd0\xbe \xd0\xbd\xd0\xb0\xd0\xbc \xd0\xb1\xd1\x83\xd0\xb4\xd0\xb5\xd1\x82 \xd0\xbd\xd0\xb5\xd0\xbe\xd0\xb1\xd1\x85\xd0\xbe\xd0\xb4\xd0\xb8\xd0\xbc\xd0\xbe \xd0\xb4\xd0\xbb\xd1\x8f \xd0\xb2\xd0\xb8\xd0\xb7\xd1\x83\xd0\xb0\xd0\xbb